In [1]:
## -- Trova il primo swing

def find_first_swing(signal):
    found = False
    swing_onset = 0
    for i in range(0,signal.shape[0]): 
        if not found:
            swing_onset += 1
        if signal[i,-1] == 1:
            found = True
    return swing_onset

In [2]:
## -- Elimina i falsi stance in mezzo agli swing - i falsi 0 in mezzo agli 1

def delete_false_stance(signal, thr, only_baso):
    cont = 0
    stop = False
    
    if only_baso:
        ## -- Trovo tutte le transizioni 1 -> 0
        transitions = []
        for i in range(0,signal.shape[0] - 1):
            if signal[i] == 1 and signal[i+1] == 0:
                transitions.append(i) 

        for k in transitions:
            non_zeros = np.count_nonzero(signal[k : k + thr + 1])
            if non_zeros != 0:
                signal[k : k + non_zeros + 1] = 1
                
    if not only_baso:
    ## -- Trovo tutte le transizioni 1 -> 0
        transitions = []
        for i in range(0,signal.shape[0] - 1):
            if signal[i,-1] == 1 and signal[i+1,-1] == 0:
                transitions.append(i) 
        
        ## -- Controllo quali siano false e le salvo
        for k in transitions:
            non_zeros = np.count_nonzero(signal[k : k + thr + 1, -1])
            if non_zeros != 0:
                signal[k : k + non_zeros + 1, -1] = 1

    return signal

In [3]:
## -- Elimina i falsi swing (non fisiologici) in base alla threshold che gli si da

# Devo trovare le altre transizioni - stance to swing - 0 -> 1
def delete_false_swing(signal, thr, only_baso):
    cont = 0
    stop = False
    
    if only_baso:
       ## -- Trovo tutte le transizioni 0 -> 1
        transitions = []
        for i in range(0,signal.shape[0] - 1):
            if signal[i] == 0 and signal[i+1] == 1:    # Se il campione i è 0 e quello i+1 è 1 --> Transizione
                transitions.append(i) 
        
        ## -- Controllo quali siano false e le salvo
        for k in transitions:
            ones = np.count_nonzero(signal[k : k + thr + 1])
            if ones < thr:
                signal[k : k + ones + 1] = 0
       
        
    if not(only_baso):
        ## -- Trovo tutte le transizioni 1 -> 0
        transitions = []
        for i in range(0,signal.shape[0] - 1):
            if signal[i,-1] == 0 and signal[i+1,-1] == 1:
                transitions.append(i) 
        
        ## -- Controllo quali siano false e le salvo
        for k in transitions:
            ones = np.count_nonzero(signal[k : k + thr + 1, -1])
            if ones < thr:
                signal[k : k + ones + 1, -1] = 0
    return signal

In [1]:
## -- Clean Basograhic Signal
# Questo script mi serve per eliminare tutti i "passi falsi" :-)

import numpy as np
from numpy import genfromtxt as csv_load
import os
import pandas as pd

thr = 100

tipo = np.dtype(np.float32)    # Per farglielo caricare 

in_directory = 'subjects/min-max/Original_Biphase_Subjects/'
prefix = 'Biphase_Subject'
suffix = '_norm.csv'
out_directory = '../subjects/min-max/clean/'
out_path_baso = out_directory + 'baso_real/'
out_prefix = 's'
baso_suffix = '_baso_real.csv'
subjects = 26

if not os.path.exists(out_directory):
    os.makedirs(out_directory)
if not os.path.exists(out_path_baso):
    os.makedirs(out_path_baso)

# for subject in [10,13,14,17]:
for subject in range(1, subjects + 1):
    # Carica il segnale (è un NUMPY)
    signal = csv_load(in_directory + prefix + str(subject) + suffix, dtype = tipo, delimiter = ',')

    # Calcola il primo swing e aggiorna il vettore
    lower_bound = find_first_swing(signal)
    signal = signal[lower_bound:,:]
    
    # Cancella le false transizioni 
    signal = delete_false_stance(signal, thr,only_baso = False)
    signal = delete_false_swing(signal, thr, only_baso = False)
    signal = delete_false_stance(signal, thr,only_baso = False)
    signal = delete_false_swing(signal, thr, only_baso = False)
    baso = signal[:,-1]
    
    signal = pd.DataFrame(signal)
    baso = pd.DataFrame(baso)
    
    if len(signal) == len(baso):
        print('Lengths match -> Saving Files...')
    else:
        print('WARNING! Subject ' + str(subject) + ' lengths do not match! Check it.')
    
    signal = signal.to_csv(out_directory + out_prefix + str(subject) + suffix, index = None, header = None)    # Salvo il file completo
    baso = baso.to_csv(out_path_baso + out_prefix + str(subject) + baso_suffix, index = None, header = None)    # Salvo solo il basografico
    
    print('Subject ' + str(subject) + ' cleaned.\n')

OSError: ../subjects/min-max/Original_Biphase_Subjects/Biphase_Subject1_norm.csv not found.

In [4]:
## -- Per i segnali predetti

import numpy as np
from numpy import genfromtxt as csv_load
# import matplotlib.pyplot as plt
import pandas as pd

tipo = np.dtype(np.float32)    # Per farglielo caricare 
#thr = 350
#thr = 200
#thr = 550
thr = 500

model = 'FF5'
folds = 23

'''
subject_to_clean = [
[20, 25, 21, 6, 4], # 1     
[4, 16, 6, 15, 5], # 2
[17, 20, 10, 2, 26], # 3
[26, 9, 17, 14, 6], # 4
[7, 6, 17, 10, 3], # 5
[2, 26, 17, 24, 13], # 6
[19, 3, 7, 15, 24], # 7
[25, 4, 6, 2, 7], # 8
[23, 8, 13, 21, 14], # 9
[9, 1, 21, 24, 3] # 10
]
'''
subject_to_clean = [
[1], # 1     
[2], # 2
[3], # 3
[5], # 4
[6], # 5
[7], # 6
[8], # 7
[9], # 8
[10], # 9
[11], # 10
[12], # 10    
[13], # 10
[14], # 10    
[15], # 10    
[16], # 10    
[17], # 10 
[18], # 10    
[19], # 10    
[20], # 10    
[21], # 10    
[22], # 10    
[23], # 10    
[26] # 10        
]


out_prefix = 's'
suffix = '_predicted_clean_' + str(thr) + '.csv'

for fold in range(0, folds):
    #in_path = 'subjects/min-max/windows_20-del_tr-False-slide-False-digits-4/folds_inter/Report_' + model + '/Fold_' + str(fold + 1) + '/'
    in_path = 'subjects/min-max/clean/windows_20-del_tr-True-slide-False-digits-3/folds_inter/Report_' + model + '/Fold_' + str(fold + 1) + '/'
    #in_path = 'subjects/min-max/clean/windows_20-del_tr-False-slide-True-digits-3-pace-1/folds_inter/Report_' + model + '/Fold_' + str(fold + 1) + '/'
    #in_path = 'subjects/min-max/clean/windows_200-del_tr-False-slide-True-digits-3-pace-20/folds_inter/Report_' + model + '/Fold_' + str(fold + 1) + '/'
    #in_path = 'subjects/bpf-20-450_rect/windows_200-del_tr-False-slide-True-digits-3-pace-20/folds_inter/Report_' + model + '/Fold_' + str(fold + 1) + '/'
    out_directory = in_path
    subjects = subject_to_clean[fold]
    for subject in subjects:
        file = 's' + str(subject) + '_predicted.csv'
        #file = 's' + str(subject) + '_predicted.csv'
        signal = csv_load(in_path + file, dtype = tipo, delimiter = ',')

        signal[0:thr] = 1

        signal = delete_false_stance(signal, thr, only_baso = True)
        signal = delete_false_swing(signal, thr, only_baso = True)
        signal = delete_false_stance(signal, thr, only_baso = True)
        signal = delete_false_swing(signal, thr, only_baso = True)

        signal

        signal = pd.DataFrame(signal)
        signal = signal.to_csv(out_directory + out_prefix + str(subject) + suffix, index = None, header = None)

In [38]:
a = [1, 2, 3, 4, 5]
a.remove(a[2])
a

[1, 2, 4, 5]